# NY市場の株価データ解析

daru, statsample, Nyaplot, IRubyを使って、株式市場のブラウン運動を観察する。
収益率の対数が正規分布に従うかどうか。

In [1]:
require "daru"
require "statsample"

"if(window['d3'] === undefined ||\n   window['Nyaplot'] === undefined){\n    var path = {\"d3\":\"https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.5/d3.min\",\"downloadable\":\"http://cdn.rawgit.com/domitry/d3-downloadable/master/d3-downloadable\"};\n\n\n\n    var shim = {\"d3\":{\"exports\":\"d3\"},\"downloadable\":{\"exports\":\"downloadable\"}};\n\n    require.config({paths: path, shim:shim});\n\n\nrequire(['d3'], function(d3){window['d3']=d3;console.log('finished loading d3');require(['downloadable'], function(downloadable){window['downloadable']=downloadable;console.log('finished loading downloadable');\n\n\tvar script = d3.select(\"head\")\n\t    .append(\"script\")\n\t    .attr(\"src\", \"http://cdn.rawgit.com/domitry/Nyaplotjs/master/release/nyaplot.js\")\n\t    .attr(\"async\", true);\n\n\tscript[0][0].onload = script[0][0].onreadystatechange = function(){\n\n\n\t    var event = document.createEvent(\"HTMLEvents\");\n\t    event.initEvent(\"load_nyaplot\",false,false);\n\t    win

true

In [2]:
# ニューヨーク市場の銘柄から100のサンプル抽出
stock_ticker = Daru::DataFrame.from_csv("stock_ticker.csv")
sample_ticker = stock_ticker.where(stock_ticker["Exchange"].eq("NYQ"))["Ticker"].to_a.shuffle.take(100)
# Daru::DataFrame.new("Ticker" => sample_ticker).write_csv("sample_ticker.csv")
# ダウンロードはpandasで:)

["HP", "BKK", "EDR", "NSS", "COO", "SBGL", "EFT", "SAR", "GM", "OGS", "SZC", "UFI", "OFC", "HAR", "TREC", "FNF", "DL", "CM", "JP", "ALDW", "NBB", "NSP", "RAS", "JNPR", "ANH", "CGA", "YZC", "SBH", "EVER", "NRF-PC", "PACD", "NMO", "IRET", "CSC", "ALU", "RLH", "CGG", "JFR", "PSA-PX", "SBY", "FGL", "EOS", "RYAM", "REN", "MGU", "DCT", "JMPC", "CRC", "BLD", "NQ", "VTR", "L", "AED", "NXC", "NEP", "TDG", "TG", "EMN", "EXTN", "AVH", "TREX", "HDB", "BSMX", "GEO", "NVO", "MCR", "CRR", "HPF", "LEJU", "RESI", "SBR", "DSM", "ANTM", "NNC", "PNY", "SFG", "DIS", "ALR", "IPI", "TWI", "DEX", "BLT", "NNA", "CI", "FOE", "BGS", "EPE", "ADM", "AFB", "EOD", "ALL-PC", "PLOW", "DSW", "NPT", "EARN", "DCI", "RGA", "HON", "WUBA", "MSI"]

## 株価データのダウンロード

* stock_ticker.csvをもとに、pandasのget_data_yahoo()を使って2006年1月1日以降の株価データを[ダウンロード](StockDownload.ipynb)
* 58銘柄分ダウンロードしたところでエラーorz（アクセス制限？）

In [3]:
# pandasでダウンロードしたCSVをロード
# 最初の日付が2006-01-03のものだけ抽出
stocks = Dir["stock/*.csv"].map { |file|
  name = file.slice(/([^\/]+)\.csv\z/, 1)
  Daru::DataFrame.from_csv(file, name: name)
}.select {|stock| stock["Date"][0] == "2006-01-03" }
stocks.size

33

In [84]:
# 直近1か月の終値の分布
recent_close = Daru::DataFrame.new(
  close: stocks.flat_map { |stock|
    stock["Adj Close"][-30..-1].to_a
  }
)
recent_close.plot type: :histogram, x: :close do |plot, diagram|
  diagram.bin_num(50)
  plot.yrange([0, 200])
end

#<Nyaplot::Frame:0x007f623891a4f0 @properties={:panes=>[#<Nyaplot::Plot:0x007f6235d82540 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007f623891aa40 @properties={:type=>:histogram, :options=>{:value=>:close, :bin_num=>50}, :data=>"9c4b52f5-2208-4012-91ef-7478db9fafcc"}, @xrange=[0, 200.5], @yrange=[0, 990]>], :options=>{:yrange=>[0, 200], :width=>700, :xrange=>[0, 200.5]}}>], :data=>{"9c4b52f5-2208-4012-91ef-7478db9fafcc"=>#<Nyaplot::DataFrame:0x007f62388fdb20 @name="9c4b52f5-2208-4012-91ef-7478db9fafcc", @rows=[{:close=>41.151256}, {:close=>41.031834}, {:close=>40.942267}, {:close=>41.519481}, {:close=>41.081593}, {:close=>41.021884}, {:close=>39.777892}, {:close=>39.628611}, {:close=>38.633418}, {:close=>37.658128000000005}, {:close=>36.971446}, {:close=>37.240145}, {:close=>37.538706}, {:close=>36.28476}, {:close=>36.891829}, {:close=>35.608028000000004}, {:close=>35.757309}, {:close=>35.29952}, {:close=>34.145096}, {:close=>34.981059}, {:close=>34.503362}, {:close=>35.389088}, {:close=>35.259712}, {:close=>35.199999}, {:close=>36.220001}, {:close=>35.5}, {:close=>34.290001000000004}, {:close=>34.34}, {:close=>34.950001}, {:close=>34.459998999999996}, {:close=>200.5}, {:close=>199.41000400000001}, {:close=>200.270004}, {:close=>200.320007}, {:close=>198.740005}, {:close=>197.229996}, {:close=>194.669998}, {:close=>195.5}, {:close=>193.100006}, {:close=>187.809998}, {:close=>182.389999}, {:close=>163.550003}, {:close=>164.009995}, {:close=>157.83999599999999}, {:close=>164.779999}, {:close=>165.0}, {:close=>169.440002}, {:close=>170.0}, {:close=>170.449997}, {:close=>170.179993}, {:close=>170.169998}, {:close=>170.990005}, {:close=>166.919998}, {:close=>160.119995}, {:close=>160.979996}, {:close=>159.270004}, {:close=>155.0}, {:close=>152.369995}, {:close=>153.809998}, {:close=>154.360001}, {:close=>21.709999}, {:close=>21.77}, {:close=>21.790001}, {:close=>21.99}, {:close=>21.74}, {:close=>21.459999}, {:close=>20.860001}, {:close=>21.74}, {:close=>21.860001}, {:close=>21.17}, {:close=>20.77}, {:close=>20.84}, {:close=>20.639999}, {:close=>20.280001000000002}, {:close=>20.32}, {:close=>19.92}, {:close=>19.83}, {:close=>19.629998999999998}, {:close=>20.07}, {:close=>20.4}, {:close=>20.01}, {:close=>20.610001}, {:close=>20.030001000000002}, {:close=>20.16}, {:close=>20.719998999999998}, {:close=>20.690001000000002}, {:close=>20.6}, {:close=>20.469998999999998}, {:close=>20.280001000000002}, {:close=>19.84}, {:close=>9.07}, {:close=>9.08}, {:close=>8.97}, {:close=>9.02}, {:close=>8.91}, {:close=>8.83}, {:close=>8.63}, {:close=>8.66}, {:close=>8.43}, {:close=>8.18}, {:close=>7.89}, {:close=>7.81}, {:close=>7.69}, {:close=>7.57}, {:close=>7.58}, {:close=>7.41}, {:close=>7.33}, {:close=>7.3}, {:close=>6.36}, {:close=>6.3}, {:close=>5.66}, {:close=>5.99}, {:close=>6.32}, {:close=>6.37}, {:close=>6.62}, {:close=>6.74}, {:close=>6.48}, {:close=>6.48}, {:close=>6.69}, {:close=>6.54}, {:close=>13.633788000000001}, {:close=>13.584210999999998}, {:close=>13.574295000000001}, {:close=>13.544549}, {:close=>13.504887}, {:close=>13.286745999999999}, {:close=>13.266916}, {:close=>13.554464999999999}, {:close=>13.633788000000001}, {:close=>13.455309}, {:close=>13.336324}, {:close=>13.257}, {:close=>13.28}, {:close=>12.92}, {:close=>12.96}, {:close=>12.84}, {:close=>12.89}, {:close=>12.66}, {:close=>12.78}, {:close=>13.02}, {:close=>13.01}, {:close=>13.08}, {:close=>13.24}, {:close=>13.39}, {:close=>13.48}, {:close=>13.62}, {:close=>13.62}, {:close=>13.68}, {:close=>13.62}, {:close=>13.56}, {:close=>82.959999}, {:close=>82.550003}, {:close=>82.449997}, {:close=>83.209999}, {:close=>82.16999799999999}, {:close=>81.410004}, {:close=>79.839996}, {:close=>79.57}, {:close=>76.91999799999999}, {:close=>73.790001}, {:close=>73.050003}, {:close=>72.889999}, {:close=>73.120003}, {:close=>71.82}, {:close=>72.07}, {:close=>70.16999799999999}, {:close=>69.290001}, {:close=>67.360001}, {:close=>68.129997}, {:close=>69.870003}, {:close=>67.879997}, {:close=

In [85]:
# 直近1か月の終値の対数の分布
recent_log_close = Daru::DataFrame.new(
  close: stocks.flat_map { |stock|
    stock["Adj Close"].tail(30).map { |x| Math.log(x) }
  }
)
recent_log_close.plot type: :histogram, x: :close do |plot, diagram|
  diagram.bin_num(50)
  plot.yrange([0, 90])
end

#<Nyaplot::Frame:0x007f6238c926c8 @properties={:panes=>[#<Nyaplot::Plot:0x007f6238acdc70 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007f6238c92c18 @properties={:type=>:histogram, :options=>{:value=>:close, :bin_num=>50}, :data=>"25a32fa6-42bb-4964-b00b-b6df2ccec25c"}, @xrange=[0, 5.300814246746624], @yrange=[0, 990]>], :options=>{:yrange=>[0, 90], :width=>700, :xrange=>[0, 5.300814246746624]}}>], :data=>{"25a32fa6-42bb-4964-b00b-b6df2ccec25c"=>#<Nyaplot::DataFrame:0x007f6238c76e78 @name="25a32fa6-42bb-4964-b00b-b6df2ccec25c", @rows=[{:close=>3.7172544491360138}, {:close=>3.7143482044558556}, {:close=>3.7121629524177115}, {:close=>3.7261627387795073}, {:close=>3.715560162302668}, {:close=>3.7141056804047508}, {:close=>3.6833112805687382}, {:close=>3.6795513573829584}, {:close=>3.65411765321963}, {:close=>3.6285488140806224}, {:close=>3.6101458849778125}, {:close=>3.6173873460640142}, {:close=>3.6253725606654874}, {:close=>3.5913978184047943}, {:close=>3.607990090211478}, {:close=>3.5725711180533923}, {:close=>3.5767546957959113}, {:close=>3.56386936611309}, {:close=>3.530618973948125}, {:close=>3.554806743574733}, {:close=>3.5410567685648044}, {:close=>3.566403524033797}, {:close=>3.5627410090407343}, {:close=>3.5610460541949602}, {:close=>3.5896114800643226}, {:close=>3.56953269648137}, {:close=>3.53485379563785}, {:close=>3.5363108554693294}, {:close=>3.553918497292122}, {:close=>3.5397992020806304}, {:close=>5.300814246746624}, {:close=>5.295363026780775}, {:close=>5.299666476090368}, {:close=>5.299916122855823}, {:close=>5.2919974629617155}, {:close=>5.284370540134521}, {:close=>5.27130580702739}, {:close=>5.27556037942542}, {:close=>5.263208220506452}, {:close=>5.235430802843786}, {:close=>5.20614724609315}, {:close=>5.097118772327407}, {:close=>5.099927371089072}, {:close=>5.061581836372241}, {:close=>5.104611244818746}, {:close=>5.10594547390058}, {:close=>5.132498893656685}, {:close=>5.135798437050262}, {:close=>5.138441980983442}, {:close=>5.136856659288492}, {:close=>5.136797925630392}, {:close=>5.141605104501977}, {:close=>5.117514643741564}, {:close=>5.075923502897792}, {:close=>5.08128010881778}, {:close=>5.070600900378275}, {:close=>5.043425116919247}, {:close=>5.026311740666404}, {:close=>5.035718061460663}, {:close=>5.03928754314778}, {:close=>3.0777729378284766}, {:close=>3.080532875246456}, {:close=>3.0814511948491385}, {:close=>3.09058780456667}, {:close=>3.0791538816930633}, {:close=>3.0661906906042287}, {:close=>3.0378334975112637}, {:close=>3.0791538816930633}, {:close=>3.0846585284940455}, {:close=>3.0525850851467737}, {:close=>3.033509637888021}, {:close=>3.0368742168851663}, {:close=>3.0272308921637485}, {:close=>3.009635228032646}, {:close=>3.011605622710281}, {:close=>2.991724252156452}, {:close=>2.9871959425317045}, {:close=>2.977058957345933}, {:close=>2.9992261628082466}, {:close=>3.0155349008501706}, {:close=>2.996232148595642}, {:close=>3.0257764434225023}, {:close=>2.997231199602838}, {:close=>3.0037004432031678}, {:close=>3.0310993691287327}, {:close=>3.0296505400899783}, {:close=>3.0252910757955354}, {:close=>3.018960350821218}, {:close=>3.009635228032646}, {:close=>2.987700101856727}, {:close=>2.2049722641270453}, {:close=>2.206074192613202}, {:close=>2.1938856760707046}, {:close=>2.199444334074532}, {:close=>2.187174241482718}, {:close=>2.178155014615869}, {:close=>2.155244505095337}, {:close=>2.1587147225743437}, {:close=>2.131796772013764}, {:close=>2.1016921506146558}, {:close=>2.065596134857783}, {:close=>2.0554049638515948}, {:close=>2.0399207835175526}, {:close=>2.0241930674493576}, {:close=>2.0255131996542803}, {:close=>2.0028304393079956}, {:close=>1.9919755158985601}, {:close=>1.9878743481543455}, {:close=>1.8500283773520307}, {:close=>1.840549633397487}, {:close=>1.7334238922150915}, {:close=>1.7900914121273581}, {:close=>1.843719208158766}, {:close=>1.851599469584072}, {:close=>1.890095369948917}, {:close=>1.9080599249242156}, {:close=>1.8687205103641833}, {:close=>1.8687205103

In [86]:
# 最初の銘柄の統計量
stocks[0].describe

,Adj Close,Close,High,Low,Open,Volume
count,2541,2541,2541,2541,2541,2541
mean,29.81528834474617,32.601987576151025,33.02706432624963,32.157398822117194,32.59627326879178,7469752.184179457
std,6.935897965268157,7.4171174553984915,7.473177477709347,7.365463820363122,7.411413090620962,4870549.229860659
min,15.902294,17.700001,18.01,15.44,17.41,326300
max,44.864678999999995,49.82,50.259997999999996,49.189999,49.799999,66396000


In [12]:
# 銘柄毎に日次収益率の対数（log(P(t + 1day) / P0(t))）を計算
log_returns = stocks.map { |stock|
  Daru::DataFrame.new(
    # [a, b, c, d, ...].each_cons(2) #=> [a, b], [b, c], [c, d], ...
    stock.each_row.each_cons(2).map { |prev, current|
      {
        date: current["Date"],
        log_return: Math.log(current["Adj Close"] / prev["Adj Close"])
      }
    }
  )
}
nil

In [11]:
# 最初の3銘柄のヒストグラムをプロット
log_returns.take(3).each do |log_return|
  log_return.plot type: :histogram, x: :log_return do |plot, diagram|
    plot.yrange([0, 500])
    diagram.bin_num(150)
  end
end
nil

#<Nyaplot::Frame:0x007f6235c1a4a0 @properties={:panes=>[#<Nyaplot::Plot:0x007f62344a0d60 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007f6235c1afb8 @properties={:type=>:histogram, :options=>{:value=>:log_return, :bin_num=>150}, :data=>"456574ae-a329-4df1-b472-96567b5a76ac"}, @xrange=[-0.3168744682392347, 0.22159435209794193], @yrange=[0, 2540]>], :options=>{:yrange=>[0, 500], :width=>700, :xrange=>[-0.3168744682392347, 0.22159435209794193]}}>], :data=>{"456574ae-a329-4df1-b472-96567b5a76ac"=>#<Nyaplot::DataFrame:0x007f6235e51ca0 @name="456574ae-a329-4df1-b472-96567b5a76ac", @rows=[{:date=>"2006-01-04", :log_return=>0.004909456575330774}, {:date=>"2006-01-05", :log_return=>0.004885577924340135}, {:date=>"2006-01-06", :log_return=>0.013012770458041365}, {:date=>"2006-01-09", :log_return=>0.005696339614212328}, {:date=>"2006-01-10", :log_return=>-0.0017952804794038875}, {:date=>"2006-01-11", :log_return=>0.010427614986358103}, {:date=>"2006-01-12", :log_return=>-0.008931376420886922}, {:date=>"2006-01-13", :log_return=>-0.0032948876223049565}, {:date=>"2006-01-17", :log_return=>-0.011770195634285163}, {:date=>"2006-01-18", :log_return=>-0.01900154819135853}, {:date=>"2006-01-19", :log_return=>-0.018423402710565483}, {:date=>"2006-01-20", :log_return=>-0.008227912943932197}, {:date=>"2006-01-23", :log_return=>0.0012788182081390081}, {:date=>"2006-01-24", :log_return=>0.003189780068425619}, {:date=>"2006-01-25", :log_return=>0.004448682033488433}, {:date=>"2006-01-26", :log_return=>0.006636134031936442}, {:date=>"2006-01-27", :log_return=>-0.005052035091319135}, {:date=>"2006-01-30", :log_return=>0.002844879941739692}, {:date=>"2006-01-31", :log_return=>0.004095221111661811}, {:date=>"2006-02-01", :log_return=>0.0018840723618869055}, {:date=>"2006-02-02", :log_return=>0.007502468456595339}, {:date=>"2006-02-03", :log_return=>0.004350529533151804}, {:date=>"2006-02-06", :log_return=>-0.0006204235198416648}, {:date=>"2006-02-07", :log_return=>0.000310277814161333}, {:date=>"2006-02-08", :log_return=>0.00803219207548322}, {:date=>"2006-02-09", :log_return=>-0.0009234918544565431}, {:date=>"2006-02-10", :log_return=>0.014371185331812147}, {:date=>"2006-02-13", :log_return=>0.011470206155610367}, {:date=>"2006-02-14", :log_return=>0.005089052238451033}, {:date=>"2006-02-15", :log_return=>0.010987513196242913}, {:date=>"2006-02-16", :log_return=>0.009991030556454896}, {:date=>"2006-02-17", :log_return=>0.011339008037589466}, {:date=>"2006-02-21", :log_return=>-0.008127789538164417}, {:date=>"2006-02-22", :log_return=>0.016476774133596862}, {:date=>"2006-02-23", :log_return=>-0.011244167371129888}, {:date=>"2006-02-24", :log_return=>-0.0026129313000415842}, {:date=>"2006-02-27", :log_return=>0.0063750302400890895}, {:date=>"2006-02-28", :log_return=>-0.011037008778763402}, {:date=>"2006-03-01", :log_return=>0.009013011768315132}, {:date=>"2006-03-02", :log_return=>0.0054842323111011625}, {:date=>"2006-03-03", :log_return=>-0.010416766785170069}, {:date=>"2006-03-06", :log_return=>-0.00495698502340355}, {:date=>"2006-03-07", :log_return=>0.0011685738728827806}, {:date=>"2006-03-08", :log_return=>0.00640282677403818}, {:date=>"2006-03-09", :log_return=>-0.015200394681671763}, {:date=>"2006-03-10", :log_return=>0.010256283231186278}, {:date=>"2006-03-13", :log_return=>0.014184750883107809}, {:date=>"2006-03-14", :log_return=>0.005732290724200348}, {:date=>"2006-03-15", :log_return=>-0.0040091971272158255}, {:date=>"2006-03-16", :log_return=>-0.004601608418854415}, {:date=>"2006-03-17", :log_return=>0.02364450483862485}, {:date=>"2006-03-20", :log_return=>-0.009618245220273002}, {:date=>"2006-03-21", :log_return=>-0.0051296804503091166}, {:date=>"2006-03-22", :log_return=>0.0008567651536864184}, {:date=>"2006-03-23", :log_return=>-0.002286417534739991}, {:date=>"2006-03-24", :log_return=>0.001429652381053465}, {:date=>"2006-03-27", :log_return=>-0.006880742457576241}, {:date=>"2006-03-28", :log_return=>-0.013613578193524996}, {:date=>"2006-

#<Nyaplot::Frame:0x007f6235ef3c08 @properties={:panes=>[#<Nyaplot::Plot:0x007f623555d3b0 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007f6235f007c8 @properties={:type=>:histogram, :options=>{:value=>:log_return, :bin_num=>150}, :data=>"6e18efca-15db-474d-a30a-1f88b7e5ea31"}, @xrange=[-0.18700803089998583, 0.1487202435589501], @yrange=[0, 2540]>], :options=>{:yrange=>[0, 500], :width=>700, :xrange=>[-0.18700803089998583, 0.1487202435589501]}}>], :data=>{"6e18efca-15db-474d-a30a-1f88b7e5ea31"=>#<Nyaplot::DataFrame:0x007f623617a328 @name="6e18efca-15db-474d-a30a-1f88b7e5ea31", @rows=[{:date=>"2006-01-04", :log_return=>0.015539056199979036}, {:date=>"2006-01-05", :log_return=>0.0016704689632160233}, {:date=>"2006-01-06", :log_return=>-0.011565118060573186}, {:date=>"2006-01-09", :log_return=>-0.02758141719841293}, {:date=>"2006-01-10", :log_return=>-0.0005787439370868549}, {:date=>"2006-01-11", :log_return=>-0.011450874754044812}, {:date=>"2006-01-12", :log_return=>0.002534383980669581}, {:date=>"2006-01-13", :log_return=>-0.0040971403340208}, {:date=>"2006-01-17", :log_return=>0.012820676633312648}, {:date=>"2006-01-18", :log_return=>0.00979548935561731}, {:date=>"2006-01-19", :log_return=>0.008753619989171452}, {:date=>"2006-01-20", :log_return=>-0.03411065296208643}, {:date=>"2006-01-23", :log_return=>-0.021199404345044876}, {:date=>"2006-01-24", :log_return=>0.0008006476969135804}, {:date=>"2006-01-25", :log_return=>0.021770171330864202}, {:date=>"2006-01-26", :log_return=>0.02034753939905907}, {:date=>"2006-01-27", :log_return=>0.0315328438813504}, {:date=>"2006-01-30", :log_return=>-0.023508376793357877}, {:date=>"2006-01-31", :log_return=>0.008526830954474693}, {:date=>"2006-02-01", :log_return=>0.011257133608252045}, {:date=>"2006-02-02", :log_return=>-0.007678667342041192}, {:date=>"2006-02-03", :log_return=>0.011774761012353542}, {:date=>"2006-02-06", :log_return=>-0.019891853447812708}, {:date=>"2006-02-07", :log_return=>-0.0005687385627018602}, {:date=>"2006-02-08", :log_return=>0.014310192841856124}, {:date=>"2006-02-09", :log_return=>-0.0018712948152327666}, {:date=>"2006-02-10", :log_return=>-0.00450531695302035}, {:date=>"2006-02-13", :log_return=>0.008244351119225286}, {:date=>"2006-02-14", :log_return=>0.014633985993541805}, {:date=>"2006-02-15", :log_return=>-0.0025777986158386466}, {:date=>"2006-02-16", :log_return=>0.0075305784530064115}, {:date=>"2006-02-17", :log_return=>-0.000915399133371064}, {:date=>"2006-02-21", :log_return=>-0.01867136669733867}, {:date=>"2006-02-22", :log_return=>0.002981189962328468}, {:date=>"2006-02-23", :log_return=>0.004455196811434798}, {:date=>"2006-02-24", :log_return=>-0.007249820626389391}, {:date=>"2006-02-27", :log_return=>0.008555066474048374}, {:date=>"2006-02-28", :log_return=>0.0024045287305171476}, {:date=>"2006-03-01", :log_return=>0.01175403732304324}, {:date=>"2006-03-02", :log_return=>-0.006410914783243988}, {:date=>"2006-03-03", :log_return=>-0.01257406380525825}, {:date=>"2006-03-06", :log_return=>-0.007283689461486893}, {:date=>"2006-03-07", :log_return=>0.0035551158534117134}, {:date=>"2006-03-08", :log_return=>0.004286641517098848}, {:date=>"2006-03-09", :log_return=>-0.014046497126686458}, {:date=>"2006-03-10", :log_return=>0.016090165533507356}, {:date=>"2006-03-13", :log_return=>-0.017222475650377527}, {:date=>"2006-03-14", :log_return=>0.006399416229097295}, {:date=>"2006-03-15", :log_return=>0.005985791477938099}, {:date=>"2006-03-16", :log_return=>0.001304625665729755}, {:date=>"2006-03-17", :log_return=>-0.0031712968004122917}, {:date=>"2006-03-20", :log_return=>0.015021152231546386}, {:date=>"2006-03-21", :log_return=>-0.00165791775225693}, {:date=>"2006-03-22", :log_return=>-0.007216283553262474}, {:date=>"2006-03-23", :log_return=>-0.019691203246957348}, {:date=>"2006-03-24", :log_return=>-0.0032248590067690693}, {:date=>"2006-03-27", :log_return=>-0.014931413643344169}, {:date=>"2006-03-28", :log_return=>-0.0052208745431975255}, {:date=>"2006-03-2

#<Nyaplot::Frame:0x007f6236231eb0 @properties={:panes=>[#<Nyaplot::Plot:0x007f623568d280 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007f6236232900 @properties={:type=>:histogram, :options=>{:value=>:log_return, :bin_num=>150}, :data=>"8e70d4ef-df1b-452c-87c5-254f242f4eb3"}, @xrange=[-0.24578605109339852, 0.2952869348435163], @yrange=[0, 2540]>], :options=>{:yrange=>[0, 500], :width=>700, :xrange=>[-0.24578605109339852, 0.2952869348435163]}}>], :data=>{"8e70d4ef-df1b-452c-87c5-254f242f4eb3"=>#<Nyaplot::DataFrame:0x007f62364797e0 @name="8e70d4ef-df1b-452c-87c5-254f242f4eb3", @rows=[{:date=>"2006-01-04", :log_return=>0.008385735958970736}, {:date=>"2006-01-05", :log_return=>0.0005217960349571437}, {:date=>"2006-01-06", :log_return=>0.00675856772589514}, {:date=>"2006-01-09", :log_return=>0.007742025277592084}, {:date=>"2006-01-10", :log_return=>0.00768242854462343}, {:date=>"2006-01-11", :log_return=>0.003056615807605268}, {:date=>"2006-01-12", :log_return=>-0.009711391197891488}, {:date=>"2006-01-13", :log_return=>-0.007216446910931098}, {:date=>"2006-01-17", :log_return=>-0.010923385471862529}, {:date=>"2006-01-18", :log_return=>0.0}, {:date=>"2006-01-19", :log_return=>0.004695931544553022}, {:date=>"2006-01-20", :log_return=>0.0020802500867174384}, {:date=>"2006-01-23", :log_return=>0.005698026574353131}, {:date=>"2006-01-24", :log_return=>0.006692340979248326}, {:date=>"2006-01-25", :log_return=>0.0005129633661540596}, {:date=>"2006-01-26", :log_return=>0.013245282916570818}, {:date=>"2006-01-27", :log_return=>0.02449506263276954}, {:date=>"2006-01-30", :log_return=>-0.016932703405491484}, {:date=>"2006-01-31", :log_return=>-0.0005023974336461171}, {:date=>"2006-02-01", :log_return=>0.0}, {:date=>"2006-02-02", :log_return=>-0.03320857562995848}, {:date=>"2006-02-03", :log_return=>-0.0005196269207730407}, {:date=>"2006-02-06", :log_return=>0.007250081479670125}, {:date=>"2006-02-07", :log_return=>0.0020619018629404913}, {:date=>"2006-02-08", :log_return=>0.014315251208033595}, {:date=>"2006-02-09", :log_return=>0.015613390671901413}, {:date=>"2006-02-10", :log_return=>0.004985164937182301}, {:date=>"2006-02-13", :log_return=>0.0004971523618018801}, {:date=>"2006-02-14", :log_return=>0.012348813866249449}, {:date=>"2006-02-15", :log_return=>0.003919625329736958}, {:date=>"2006-02-16", :log_return=>-0.00391962532973709}, {:date=>"2006-02-17", :log_return=>0.006849379423822895}, {:date=>"2006-02-21", :log_return=>-0.00048769666347535075}, {:date=>"2006-02-22", :log_return=>0.0058366083072170834}, {:date=>"2006-02-23", :log_return=>-0.0009704241312742992}, {:date=>"2006-02-24", :log_return=>0.002908339869762172}, {:date=>"2006-02-27", :log_return=>0.012506228467506687}, {:date=>"2006-02-28", :log_return=>-0.03205717212630041}, {:date=>"2006-03-01", :log_return=>0.013238796638873932}, {:date=>"2006-03-02", :log_return=>-0.019676083191111447}, {:date=>"2006-03-03", :log_return=>0.003471441179073241}, {:date=>"2006-03-06", :log_return=>-0.014460574177495928}, {:date=>"2006-03-07", :log_return=>-0.0025143985862218442}, {:date=>"2006-03-08", :log_return=>-0.00555417275216111}, {:date=>"2006-03-09", :log_return=>-0.006094499817539903}, {:date=>"2006-03-10", :log_return=>0.0015271400417328225}, {:date=>"2006-03-13", :log_return=>0.01514402335351668}, {:date=>"2006-03-14", :log_return=>-0.001504053948133924}, {:date=>"2006-03-15", :log_return=>0.0030059647822239213}, {:date=>"2006-03-16", :log_return=>0.03151419146638474}, {:date=>"2006-03-17", :log_return=>0.0033874478533491917}, {:date=>"2006-03-20", :log_return=>-0.018330971038171016}, {:date=>"2006-03-21", :log_return=>-0.017654943054735243}, {:date=>"2006-03-22", :log_return=>-0.006125519610994238}, {:date=>"2006-03-23", :log_return=>0.005107182505572542}, {:date=>"2006-03-24", :log_return=>0.018173151646862354}, {:date=>"2006-03-27", :log_return=>-0.010055344093193948}, {:date=>"2006-03-28", :log_return=>-0.001517077768060776}, {:date=>"2006-03-29", :log_return=>0.014569447879251

In [95]:
# 標本の個々の値から計算した経験累積分布と、標本平均・標本標準偏差から
# 理論的に計算した正規累積分布をプロットして比較

def empirical_cumulative_distribution(v)
  x = []
  count = []
  # 簡便さのためsortを使う
  v.sort.each do |i|
    if x.empty? || x[-1] < i
      # xが空またはxの最後の要素がiより小さい時
      x.push(i)
      count.push((count[-1] || 0) + 1) # 最後のcount + 1を追加
    else
      # xの最後の値とiが同じ時
      count[-1] += 1 # 最後のcountをincrement
    end
  end
  # countの値をvの総数で割った値をy (0〜1)とする
  y = count.map { |i| i / v.count.to_f }
  Daru::DataFrame.new(x: x, y: y)
end

def normal_cumulative_distribution(v)
  x = v.min.step(v.max, (v.max - v.min) / 99).to_a
  y = x.map { |i|
    Distribution::Normal.cdf((i - v.mean) / v.std)
  }
  Daru::DataFrame.new(x: x, y: y)
end

log_returns.take(3).each do |log_return|
  v = log_return[:log_return]
  ecd = empirical_cumulative_distribution(v)
  ncd = normal_cumulative_distribution(v)
  plot = Nyaplot::Plot.new
  colors = Nyaplot::Colors.qual
  plot.add_with_df(ecd, :line, :x, :y)
  plot.add_with_df(ncd, :line, :x, :y).color("#44bb44")
  plot.xrange([-0.1, 0.1])
  plot.show
end
nil

#<Nyaplot::Frame:0x007f62385841b0 @properties={:panes=>[#<Nyaplot::Plot:0x007f6238585bc8 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007f62385853a8 @properties={:type=>:line, :options=>{:x=>:x, :y=>:y}, :data=>"7b4e926e-342a-4329-b751-9b190454f61f"}, @xrange=[-0.3168744682392347, 0.22159435209794193], @yrange=[0.0003937007874015748, 1.0]>, #<Nyaplot::Diagram:0x007f6238584ae8 @properties={:type=>:line, :options=>{:x=>:x, :y=>:y, :color=>"#44bb44"}, :data=>"cf9d7555-2892-4aa5-a285-accb527439fb"}, @xrange=[-0.3168744682392347, 0.22159435209794187], @yrange=[5.69356708855252e-33, 1.0]>], :options=>{:xrange=>[-0.1, 0.1], :zoom=>true, :width=>700, :yrange=>[5.69356708855252e-33, 1.0]}}>], :data=>{"7b4e926e-342a-4329-b751-9b190454f61f"=>
#<Daru::DataFrame:70029922245040 @name = 7b4e926e-342a-4329-b751-9b190454f61f @size = 2517>
                    x          y 
         0 -0.3168744 0.00039370 
         1 -0.1893145 0.00078740 
         2 -0.1725877 0.00118110 
         3 -0.1715250 0.00157480 
         4 -0.1558623 0.00196850 
         5 -0.1311873 0.00236220 
         6 -0.1204489 0.00275590 
         7 -0.1123763 0.00314960 
         8 -0.1116004 0.00354330 
         9 -0.1069879 0.00393700 
        10 -0.1058566 0.00433070 
        11 -0.1024881 0.00472440 
        12 -0.1017107 0.00511811 
        13 -0.1014299 0.00551181 
        14 -0.1007028 0.00590551 
       ...        ...        ... 
, "cf9d7555-2892-4aa5-a285-accb527439fb"=>
#<Daru::DataFrame:70029913584000 @name = cf9d7555-2892-4aa5-a285-accb527439fb @size = 100>
                    x          y 
         0 -0.3168744 5.69356708 
         1 -0.3114353 6.45168688 
         2 -0.3059963 7.01405060 
         3 -0.3005572 7.31603275 
         4 -0.2951181 7.32146695 
         5 -0.2896790 7.02977003 
         6 -0.2842399 6.47604352 
         7 -0.2788009 5.72412871 
         8 -0.2733618 4.85451301 
         9 -0.2679227 3.95024990 
        10 -0.2624836 3.08427015 
        11 -0.2570445 2.31065908 
        12 -0.2516055 1.66104746 
        13 -0.2461664 1.14577257 
        14 -0.2407273 7.58390688 
       ...        ...        ... 
}, :extension=>[]}>

#<Nyaplot::Frame:0x007f62382b0ab0 @properties={:panes=>[#<Nyaplot::Plot:0x007f62382b2590 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007f62382b1d20 @properties={:type=>:line, :options=>{:x=>:x, :y=>:y}, :data=>"94917e8c-4791-446d-aca6-dd5697bca288"}, @xrange=[-0.18700803089998583, 0.1487202435589501], @yrange=[0.0003937007874015748, 1.0]>, #<Nyaplot::Diagram:0x007f62382b1438 @properties={:type=>:line, :options=>{:x=>:x, :y=>:y, :color=>"#44bb44"}, :data=>"c728576c-b34c-4ab9-9b94-494845e73762"}, @xrange=[-0.18700803089998583, 0.1487202435589501], @yrange=[2.835072427852648e-29, 1.0]>], :options=>{:xrange=>[-0.1, 0.1], :zoom=>true, :width=>700, :yrange=>[2.835072427852648e-29, 1.0]}}>], :data=>{"94917e8c-4791-446d-aca6-dd5697bca288"=>
#<Daru::DataFrame:70029914388140 @name = 94917e8c-4791-446d-aca6-dd5697bca288 @size = 2532>
                    x          y 
         0 -0.1870080 0.00039370 
         1 -0.1137498 0.00078740 
         2 -0.1120166 0.00118110 
         3 -0.1090284 0.00157480 
         4 -0.0936946 0.00196850 
         5 -0.0869593 0.00236220 
         6 -0.0731600 0.00275590 
         7 -0.0678255 0.00314960 
         8 -0.0657815 0.00354330 
         9 -0.0639877 0.00393700 
        10 -0.0617723 0.00433070 
        11 -0.0602013 0.00472440 
        12 -0.0576174 0.00511811 
        13 -0.0557696 0.00551181 
        14 -0.0555750 0.00590551 
       ...        ...        ... 
, "c728576c-b34c-4ab9-9b94-494845e73762"=>
#<Daru::DataFrame:70029912979700 @name = c728576c-b34c-4ab9-9b94-494845e73762 @size = 100>
                    x          y 
         0 -0.1870080 2.83507242 
         1 -0.1836168 2.70326109 
         2 -0.1802256 2.47523675 
         3 -0.1768344 2.17648403 
         4 -0.1734432 1.83784789 
         5 -0.1700520 1.49033749 
         6 -0.1666608 1.16061086 
         7 -0.1632696 8.68004305 
         8 -0.1598784 6.23443410 
         9 -0.1564872 4.30050219 
        10 -0.1530960 2.84902178 
        11 -0.1497048 1.81273692 
        12 -0.1463136 1.10775737 
        13 -0.1429224 6.50181139 
        14 -0.1395313 3.66533356 
       ...        ...        ... 
}, :extension=>[]}>

#<Nyaplot::Frame:0x007f6239344fe8 @properties={:panes=>[#<Nyaplot::Plot:0x007f6239346a00 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007f62393461e0 @properties={:type=>:line, :options=>{:x=>:x, :y=>:y}, :data=>"462cde52-e840-4ee0-b1ac-7baf2204e414"}, @xrange=[-0.24578605109339852, 0.2952869348435163], @yrange=[0.0003937007874015748, 1.0]>, #<Nyaplot::Diagram:0x007f6239345920 @properties={:type=>:line, :options=>{:x=>:x, :y=>:y, :color=>"#44bb44"}, :data=>"6aa2c2e6-6311-40c5-86b3-cad41c2704ac"}, @xrange=[-0.24578605109339852, 0.29528693484351626], @yrange=[8.178808725847022e-16, 1.0]>], :options=>{:xrange=>[-0.1, 0.1], :zoom=>true, :width=>700, :yrange=>[8.178808725847022e-16, 1.0]}}>], :data=>{"462cde52-e840-4ee0-b1ac-7baf2204e414"=>
#<Daru::DataFrame:70029877349360 @name = 462cde52-e840-4ee0-b1ac-7baf2204e414 @size = 2487>
                    x          y 
         0 -0.2457860 0.00039370 
         1 -0.2172211 0.00078740 
         2 -0.2069204 0.00118110 
         3 -0.2053112 0.00157480 
         4 -0.1997603 0.00196850 
         5 -0.1865056 0.00236220 
         6 -0.1541505 0.00275590 
         7 -0.1367172 0.00314960 
         8 -0.1286174 0.00354330 
         9 -0.1283094 0.00393700 
        10 -0.1202859 0.00433070 
        11 -0.1165197 0.00472440 
        12 -0.1140242 0.00511811 
        13 -0.1129809 0.00551181 
        14 -0.1126629 0.00590551 
       ...        ...        ... 
, "6aa2c2e6-6311-40c5-86b3-cad41c2704ac"=>
#<Daru::DataFrame:70029920630940 @name = 6aa2c2e6-6311-40c5-86b3-cad41c2704ac @size = 100>
                    x          y 
         0 -0.2457860 8.17880872 
         1 -0.2403206 3.36776879 
         2 -0.2348552 1.34465170 
         3 -0.2293899 5.20596730 
         4 -0.2239245 1.95445862 
         5 -0.2184591 7.11532716 
         6 -0.2129937 2.51198595 
         7 -0.2075283 8.60012542 
         8 -0.2020629 2.85541801 
         9 -0.1965975 9.19443125 
        10 -0.1911322 2.87133704 
        11 -0.1856668 8.69686013 
        12 -0.1802014 2.55491111 
        13 -0.1747360 7.28016656 
        14 -0.1692706 2.01222992 
       ...        ...        ... 
}, :extension=>[]}>